<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Quantum Circuits

The `QuantumCircuit`, `QuantumRegister`, and `ClassicalReigster` are the main objects for Qiskit Terra. Most users will be able to do all they want with these objects. 

NOTE THIS ONLY WORKS ON QISKIT 0.7


In [1]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

## Quantum and Classical Registers

Quantum and Classical Registers are declared using the following:

In [2]:
q0 = QuantumRegister(2, 'q0')
c0 = ClassicalRegister(2, 'c0')
q1 = QuantumRegister(2, 'q1')
c1 = ClassicalRegister(2, 'c1')
q_test = QuantumRegister(2, 'q0')

The name is optional. If not given Qiskit will name it $qi$ where $i$ is an interger which will count from 0. The name and size can be returned using the following:

In [3]:
print(q0.name)
print(q0.size)

q0
2


You can test if the register are the same or different. 

In [4]:
q0==q0

True

In [5]:
q0==q_test

True

In [6]:
q0==q1

False

## Quantum Circuits

Quantum Circuits are made using registers. Either when initiated or by using the `add_register` command. 

In [7]:
circ = QuantumCircuit(q0, q1)
circ.x(q0[1])
circ.x(q1[0])
circ.draw()

is the same as 

In [8]:
circ2 = QuantumCircuit()
circ2.add_register(q0)
circ2.add_register(q1)
circ2.x(q0[1])
circ2.x(q1[0])
circ2.draw()

<div class="alert alert-block alert-info">
<b>Note:</b> The order in the list is the order they are declared (Not tensor product).
</div>

In [9]:
from copy import deepcopy

q3 = QuantumRegister(2, 'q3')
circ3 = deepcopy(circ)
circ3.add_register(q3)
circ3.draw()

<div class="alert alert-block alert-info">
<b>Note:</b> The circuit drawer has the last register added at the top and if we add a new register it will add it to the top of the circuit. 
</div>

CHECK AFTER UPDATE IS CORRECT

### Extending a circuit

In many situations you will may have two circuits that you want to add together to form a new circuit. This is very useful when one circuit has no measurements and the final circuit represents a measurement.  

In [10]:
meas = QuantumCircuit(q0, q1, c0, c1)
meas.measure(q0, c0)
meas.measure(q1, c1)

qc = circ + meas

qc.draw()

In [11]:
meas2 = QuantumCircuit()
meas2.add_register(q0)
meas2.add_register(q1)
meas2.add_register(c0)
meas2.add_register(c1)
meas2.measure(q0, c0)
meas2.measure(q1, c1)

qc2 = circ2 + meas2

qc2.draw()

It even works when the circuits have different registors. Lets start by making two new circuits

In [12]:
circ4 = QuantumCircuit(q1)
circ4.x(q1)
circ4.draw()

In [13]:
circ5 = QuantumCircuit(q3)
circ5.h(q3)
circ5.draw()

The new register is added to the circuit (top)

In [14]:
(circ4+circ5).draw()

We have also overloaded `+=` to the `QuantumCircuit` object.

In [15]:
circ4 += circ5
circ4.draw()

## Outcomes of Quantum Circuits

The MSB is to the left and the LSB is to the right. In this example 

In [16]:
circ.draw()

Qubit register $Q_0$ is prepared in the state $|10\rangle$ and $Q_1$ is in the state $|01\rangle$ giving a total state $|0110\rangle$ ($Q1\otimes Q0$). 

<div class="alert alert-block alert-info">
<b>Note:</b> The tensor order in Qiskit goes $Q_n \otimes .. Q_1 \otimes Q_0$
</div>

That is the fout qubit state is a vector of length 16 with the 6th element (`int('0110',2)=6`) being one. Note count from zero.   

In [17]:
backend_sim = Aer.get_backend('statevector_simulator')
result = execute(circ, backend_sim).result()
state = result.get_statevector(circ)
print(state)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


To check the fidelity of this state with the `basis_state` in Qiskit Terra you can use:

In [18]:
state_fidelity(basis_state('0110', 4), state)

1.0

We can also use Qiskit Terra to make the unitary operator representing the circuit (provided there is no measurments). This will be a 16x16 matrix equal to $I\otimes X\otimes X\otimes I$. To check this is correct we can use the `Pauli` class and the `process_fidelity` function. 

In [19]:
backend_sim = Aer.get_backend('unitary_simulator')
result = execute(circ, backend_sim).result()
unitary = result.get_unitary(circ)
process_fidelity(Pauli(label='IXXI').to_matrix(), unitary)

1.0

To map the information of the quantum state to the classial world we have to use the example with measurements `qc`

In [20]:
qc.draw()

This will map the quantum state to the classical world and since state has no superpositions it will be deterministic and equal to `'01 10'`.  Here a space is used to separate the registers.

In [21]:
backend_sim = Aer.get_backend('qasm_simulator')
result = execute(qc, backend_sim).result()
counts = result.get_counts(qc)
print(counts)

{'01 10': 1024}


To show that it does not matter how you add the registers we run the same as above on the second example 

In [22]:
backend_sim = Aer.get_backend('statevector_simulator')
result = execute(circ2, backend_sim).result()
states = result.get_statevector(circ2)

backend_sim = Aer.get_backend('qasm_simulator')
result = execute(qc2, backend_sim).result()
counts = result.get_counts(qc2)

backend_sim = Aer.get_backend('unitary_simulator')
result = execute(circ2, backend_sim).result()
unitary = result.get_unitary(circ2)

In [23]:
print(counts)

{'01 10': 1024}


In [24]:
state_fidelity(basis_state('0110', 4), state)

1.0

In [25]:
process_fidelity(Pauli(label='IXXI').to_matrix(), unitary)

1.0